# Combining Everything!

## Imports and Setting Up

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import pickle
import matplotlib.pyplot as plt
import os
import cv2
from tqdm.autonotebook import tqdm
from keras import layers
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import mean_squared_error

import torch
import math
from torch import nn

In [2]:
%matplotlib inline

## Loading the data and preprocessing it

In [3]:
with open("../input/weather-data/data.pkl", "rb") as f:
    l = pickle.load(f)

The function below resizes the image using a custom resizer (since cv2 resize was just jank)

In [4]:
def mean_conv(img):
    size = 8
    stride = 6
    ret = np.ones((5,5))
    stx=0
    for i in range(5):
        sty = 0
        for j in range(5):
            ret[i][j] = np.mean(img[stx:stx+size, sty:sty+size])
            sty += stride
        stx += stride
    
    return ret

In [5]:
SIZE = 32
high_img = []
for i in tqdm(range(len(l))):
    img = l[i]
    img[np.isnan(img)]=np.mean(img[~np.isnan(img)])
    if (i!=0 and np.array_equal(img, previmg)):
        continue
    previmg = img
    #resizing image
    img = cv2.resize(img, (SIZE, SIZE))
    img = img / img.max()
    high_img.append(img)

Too many images are duplicates, so let's fix that!

In [6]:
high_img = np.unique(np.array(high_img), axis=0)
high_img.shape

Create the low images (which us bicubic interpolation to scale up to the preferred dimensions)

In [7]:
temp_img = []
low_img = []
for i in tqdm(range(len(high_img))):
    img = high_img[i]
    #resizing image
    img = cv2.resize(img, (SIZE, SIZE))
    temp = mean_conv(img)
    temp_img.append(temp)
    img = cv2.resize(temp, (8, 8), interpolation=cv2.INTER_CUBIC)
    low_img.append(img)

In [8]:
high_img = np.array(high_img)
low_img = np.array(low_img)
temp_img = np.array(temp_img)

## Model Creation

The core of the model is the UNetBlock, full of deconvolutions and upconvolutions.

In [ ]:
class UNetBlock(keras.layers.Layer):
    def __init__(self):
        super(UNetBlock, self).__init__()
        self.down1 = self.down(128,(3,3),False)
        self.down2 = self.down(128,(3,3),False)
        self.down3 = self.down(256,(3,3),False)
#         self.down4 = self.down(512,(3,3),False)
        
#         self.up1 = self.up(256,(3,3),False)
        self.up2 = self.up(128,(3,3),False)
        self.up3 = self.up(128,(3,3),False)
        self.up4 = self.up(64,(3,3),False)
        
        self.out = layers.Conv2D(8, (3,3), padding = 'same', activation='relu')
        
    def call(self, inputs):
        d1 = self.down1(inputs)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        
        u2 = self.up2(d3)
        u2 = layers.concatenate([u2, d2])
        u3 = self.up3(u2)
        u3 = layers.concatenate([u3, d1])
        u4 = self.up4(u3)
        u4 = layers.concatenate([u4, inputs])
        
        output = self.out(u4)
        return output
        
    def down(self, filters , kernel_size, apply_batch_normalization = True):
        downsample = tf.keras.models.Sequential()
        downsample.add(layers.Conv2D(filters,kernel_size,padding = 'same', strides = 2, activation='relu'))
        if apply_batch_normalization:
            downsample.add(layers.BatchNormalization())
        downsample.add(keras.layers.LeakyReLU())
        return downsample
    
    def up(self, filters, kernel_size, dropout = False):
        upsample = tf.keras.models.Sequential()
        upsample.add(layers.Conv2DTranspose(filters, kernel_size,padding = 'same', strides = 2, activation='relu'))
        if dropout:
            upsample.dropout(0.2)
        upsample.add(keras.layers.LeakyReLU())
        return upsample

In [10]:
def unet_model(size):
    model = keras.Sequential()
    model.add(layers.Input(shape=[size,size,1]))
    model.add(UNetBlock())
    model.add(layers.Conv2D(1, (1,1), padding='same', activation='relu'))
    
    return model

We create the models with empty weights (intially)

In [11]:
m5_8 = unet_model(8)

In [12]:
class UNetBlock(keras.layers.Layer):
    def __init__(self):
        super(UNetBlock, self).__init__()
        self.down1 = self.down(128,(3,3),False)
        self.down2 = self.down(128,(3,3),False)
        self.down3 = self.down(256,(3,3),False)
        self.down4 = self.down(512,(3,3),False)
        
        self.up1 = self.up(256,(3,3),False)
        self.up2 = self.up(128,(3,3),False)
        self.up3 = self.up(128,(3,3),False)
        self.up4 = self.up(64,(3,3),False)
        
        self.out = layers.Conv2D(8, (3,3), padding = 'same', activation='relu')
        
    def call(self, inputs):
        d1 = self.down1(inputs)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        
        u1 = self.up1(d4)
        u1 = layers.concatenate([u1, d3])
        u2 = self.up2(u1)
        u2 = layers.concatenate([u2, d2])
        u3 = self.up3(u2)
        u3 = layers.concatenate([u3, d1])
        u4 = self.up4(u3)
        u4 = layers.concatenate([u4, inputs])
        
        output = self.out(u4)
        return output
        
    def down(self, filters , kernel_size, apply_batch_normalization = True):
        downsample = tf.keras.models.Sequential()
        downsample.add(layers.Conv2D(filters,kernel_size,padding = 'same', strides = 2, activation='relu'))
        if apply_batch_normalization:
            downsample.add(layers.BatchNormalization())
        downsample.add(keras.layers.LeakyReLU())
        return downsample
    
    def up(self, filters, kernel_size, dropout = False):
        upsample = tf.keras.models.Sequential()
        upsample.add(layers.Conv2DTranspose(filters, kernel_size,padding = 'same', strides = 2, activation='relu'))
        if dropout:
            upsample.dropout(0.2)
        upsample.add(keras.layers.LeakyReLU())
        return upsample

In [13]:
m8_16 = unet_model(16)
m16_32 = unet_model(32)

We load the pretrained weights into the model (the notebook was run in kaggle for free gpu)

In [14]:
m5_8.load_weights('../input/model-saves-impactful-weather/5_8/md_5_8_best')
m8_16.load_weights('../input/model-saves-impactful-weather/8_16/md_8_16_best')
m16_32.load_weights('../input/model-saves-impactful-weather/16_32/md_16_32_best')

Let's run a sample prediction to see how it does!

In [36]:
pred1 = m5_8.predict(low_img[2].reshape(1, 8, 8, 1))
pred1 = np.clip(pred1, 0.0, 1.0)
pred1 = pred1.reshape(8, 8, 1)
pred2 = cv2.resize(pred1, (16,16))
pred2 = m8_16.predict(pred2.reshape(1, 16, 16, 1))
pred2 = np.clip(pred2, 0.0, 1.0)
pred2 = pred2.reshape(16, 16, 1)
pred3 = cv2.resize(pred2, (32,32))
pred3 = m16_32.predict(pred3.reshape(1, 32, 32, 1))
pred3 = np.clip(pred3, 0.0, 1.0)
pred3 = pred3.reshape(32, 32, 1)

In [37]:
plt.imshow(low_img[2])
plt.show()
plt.imshow(pred1)
plt.show()
plt.imshow(pred2)
plt.show()
plt.imshow(pred3)
plt.show()
plt.imshow(high_img[2])
plt.show()

The following function calculates the RMSE score given the index of the function

In [21]:
def rmse(index):
    pred1 = m5_8.predict(low_img[index].reshape(1, 8, 8, 1))
    pred1 = np.clip(pred1, 0.0, 1.0)
    pred1 = pred1.reshape(8, 8, 1)
    pred2 = cv2.resize(pred1, (16,16))
    pred2 = m8_16.predict(pred2.reshape(1, 16, 16, 1))
    pred2 = np.clip(pred2, 0.0, 1.0)
    pred2 = pred2.reshape(16, 16, 1)
    pred3 = cv2.resize(pred2, (32,32))
    pred3 = m16_32.predict(pred3.reshape(1, 32, 32, 1))
    pred3 = np.clip(pred3, 0.0, 1.0)
    pred3 = pred3.reshape(32, 32, 1)
    return np.sqrt(mean_squared_error(high_img[index].reshape(1024), pred3.reshape(1024)))

In [22]:
rmse_sum = 0
for i in tqdm(range(1984)):
    rmse_sum += rmse(i)

rmse_sum /= 1984
rmse_sum